In [ ]:
!{sys.executable} -m conda install astropy

In [1]:
from utils import Candidate
from utils import Quadrant
import pandas as pd
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
import astropy.wcs as wcs

import psf
import crowdsource_base

from crowdsource_functions import utils

### Goal:
1. Do Dr. Knop's crowdsource on the reference image. Make sure the model and subtraction are much better.
2. Generate a reference csv file.
3. Based on the reference csv file, do Dr. Knop's crowdsource on another random image. Make sure the model and subtraction are much better.
4. Experiment with using gaussian vs. moffat psfs and toggling the refit_sky and other parameters on and off.

In [2]:
# Get the reference and test images

name = "DC21saaw"
radec = (270.31184, -28.77637)
c = Candidate(name, radec)
c.start()
iid = c.get_image_ids()
imref_fp, wref_fp = c.get_fp("c4d_210322_091756_ori") # Tuple of image and weight file
imfnames, wfnames = zip(*c.get_all_fps()) # List of tuples of image and weight files
c.close()

## Part 1: Crowdsource on a Reference Image

In [ ]:
# Define tile parameters

s = 32 # Half the length of the picture in pixels
radec = (270.31177, -28.776405) # Given Ra/Dec to center on
radec = (270.31178, -28.77640) # Ra/Dec updated manually (why is it offset originally tho?)
imhdul = fits.open(imref_fp) # get an image reference file
weighthdul = fits.open(wref_fp) # get its weight reference file

In [ ]:
# Create a quadrant and get image tile
q = Quadrant(imhdul, (2*s, 2*s)) # start a tile
subim, c = q.get_tile((radec[0], radec[1])) # Get the image centered on the int near radect
ymin, ymax, xmin, xmax = c #Boundaries of tile
weight = utils.get_weight(weighthdul, c) # Get the weight tile

In [ ]:
# Run Crowdsource
psf_ = psf.gaussian_psf(4, deriv=False) # get the psf stamps
psf_ = np.reshape(psf_, (19, 19)) # reshape the psf into the shape the SimplePSF object needs.
psf__ = psf.SimplePSF(psf_) # make a SimplePSF psf object with the stamps.
    
#CROWDSOURCE GET XS AND YS
pars = crowdsource_base.fit_im(subim, psf__, weight=weight, 
                           verbose=True, miniter=4, maxiter=10, 
                           refit_psf=True, derivcentroids=True, refit_sky = False) # Get the crowdsource fit (including sky subtraction).

In [ ]:
# Get the full positions (crowdsource positions go from 0 to 2*s)
xref, yref = utils.get_full_pos(pars, s, c)
radecs = utils.get_radecs(imhdul, xref, yref)

In [ ]:
# Check everything by plotting
print("starting reference image")
plt.imshow(subim, vmin=-1600, vmax=1600 )
plt.scatter(pars[0]["y"], pars[0]["x"], s=50, color = "red", alpha = 1, marker = "x")
plt.show()

print("reference model image")
plt.imshow(pars[1], vmin=-1600, vmax=1600 )
plt.scatter(pars[0]["y"], pars[0]["x"], s=50, color = "red", alpha = 1, marker = "x")
plt.show()

print("reference difference image")
plt.imshow(subim - pars[1], vmin=-1600, vmax=1600 )
plt.show()

## Part 2: Forced Fit Crowdsource on an Arbitrary Image

In [ ]:
def get_ts_row(imfname, wfname, csdict):
    try:
        imhdul = fits.open(imfname) # get a file
        weighthdul = fits.open(wfname)

        q = Quadrant(imhdul, (2*s, 2*s)) # start a tile
        subim, c = q.get_tile((radec[0], radec[1])) # Get the image centered on the int near radect
        ymin, ymax, xmin, xmax = c #Boundaries of tile
        weight = utils.get_weight(weighthdul, c) # Get the weight tile

        x, y = utils.get_xys(imhdul, radecs) # Get X/Y positions from ra/decs
        xref, yref = utils.get_rel_pos(x, y, s, c) # Get relative positions in terms of tile

        # Run Crowdsource Forced Fit
        psf_ = psf.gaussian_psf(4, deriv=False) # get the psf stamps
        psf_ = np.reshape(psf_, (19, 19)) # reshape the psf into the shape the SimplePSF object needs.
        psf__ = psf.SimplePSF(psf_) # make a SimplePSF psf object with the stamps.

        psf__ = psf.MoffatPSF(4, beta=3)

        #CROWDSOURCE GET XS AND YS
        pars_ = crowdsource_base.fit_im_force(subim, 
                                         yref, xref, 
                                         psf__, weight = weight, psfderiv = False, derivcentroids=False, refit_psf=True, refit_sky = True, startsky = 0) # Do the forced fit



        x = pars_[0]["y"]
        y = pars_[0]["x"]

        # Get Center Pos from Ra/Dec
        xy = q.get_xy_from_radec(np.array([[radec[0], radec[1]]])) # convert radec to pixel values
        xcenter = xy[0][0] - ymin
        ycenter = xy[0][1] - xmin

        xres = abs(x - xcenter)
        yres = abs(y - ycenter)

        totres = list(np.sqrt(xres**2 + yres**2))

        index = totres.index(min(totres))

        xpos = x[index]
        ypos = y[index]


        # Now get ra/dec, x, y, mjd, flux, dx, dy, and dflux for this
        mjd = imhdul[1].header["MJD-OBS"]
        ra = radec[0]
        dec = radec[1]
        flux = pars_[0]["flux"][index]
        dflux = pars_[0]["dflux"][index]
        dx = pars_[0]["dy"][index]
        dy = pars_[0]["dx"][index]

        csdict["MJD"].append(mjd)
        csdict["RA"].append(ra)
        csdict["DEC"].append(dec)
        csdict["flux"].append(flux)
        csdict["dflux"].append(dflux)
        csdict["x"].append(xpos)
        csdict["y"].append(ypos)
        csdict["dx"].append(dx)
        csdict["dy"].append(dy)
        csdict["sub stdv"].append(np.std(subim-pars[1]))

        return csdict
    
    except (OSError, FileNotFoundError) as e:
        return csdict

In [ ]:
cs_dict = {"MJD":[], "RA":[], "DEC":[], "flux":[], "dflux":[], "x":[], "y":[], "dx":[], "dy":[], "sub stdv": []}
for i in range(len(imfnames)):
    cs = get_ts_row(imfnames[i], wfnames[i], cs_dict)
    
print(cs_dict)

In [ ]:
test_df = pd.DataFrame(cs_dict)
print(test_df)

In [ ]:
test_df.to_csv("test.csv")

In [ ]:
imhdul = fits.open(imfnames[2]) # get a file
weighthdul = fits.open(wfnames[2])

q = Quadrant(imhdul, (2*s, 2*s)) # start a tile
subim, c = q.get_tile((radec[0], radec[1])) # Get the image centered on the int near radect
ymin, ymax, xmin, xmax = c #Boundaries of tile
weight = utils.get_weight(weighthdul, c) # Get the weight tile

In [ ]:
imhdul = fits.open(imfnames[2]) # get a file
weighthdul = fits.open(wfnames[2])

q = Quadrant(imhdul, (2*s, 2*s)) # start a tile
subim, c = q.get_tile((radec[0], radec[1])) # Get the image centered on the int near radect
ymin, ymax, xmin, xmax = c #Boundaries of tile
weight = utils.get_weight(weighthdul, c) # Get the weight tile

In [ ]:
x, y = utils.get_xys(imhdul, radecs) # Get X/Y positions from ra/decs
xref, yref = utils.get_rel_pos(x, y, s, c) # Get relative positions in terms of tile

In [ ]:
# Run Crowdsource Forced Fit
psf_ = psf.gaussian_psf(4, deriv=False) # get the psf stamps
psf_ = np.reshape(psf_, (19, 19)) # reshape the psf into the shape the SimplePSF object needs.
psf__ = psf.SimplePSF(psf_) # make a SimplePSF psf object with the stamps.

psf__ = psf.MoffatPSF(4, beta=3)
    
#CROWDSOURCE GET XS AND YS
pars_ = crowdsource_base.fit_im_force(subim, 
                                 yref, xref, 
                                 psf__, weight = weight, psfderiv = False, derivcentroids=False, refit_psf=True, refit_sky = True, startsky = 0) # Do the forced fit

In [ ]:
# Plot everything to make sure it looks ok
print("starting reference image")
plt.imshow(subim, vmin=-1600, vmax=1600 )
plt.scatter(pars[0]["y"], pars[0]["x"], s=50, color = "red", alpha = 1, marker = "x")
plt.show()

print("reference model image")
plt.imshow(pars_[1], vmin=-1600, vmax=1600 )
plt.scatter(pars_[0]["y"], pars_[0]["x"], s=50, color = "red", alpha = 1, marker = "x")
plt.show()

print("reference difference image")
plt.imshow(subim - pars_[1], vmin=-1600, vmax=1600 )
plt.show()

In [6]:
# It does look ok!! (Finally)
# Get bands for the images
bands = []
for i in range(len(imfnames)):
    try:
        hdul = fits.open(imfnames[i])
        bands.append(hdul[1].header["FILTER"][0])
        
    except (OSError, FileNotFoundError) as e:
        print("Corrupt/No File")

Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/No File
Corrupt/

In [7]:
print(len(bands), bands[0:20])

1312 ['g', 'r', 'i', 'z', 'g', 'r', 'i', 'z', 'g', 'r', 'i', 'z', 'g', 'r', 'i', 'z', 'g', 'r', 'i', 'z']


In [11]:
df = pd.read_csv("test.csv")
df["band"] = bands
df.to_csv("test.csv")

In [12]:
pd.read_csv("test.csv")

,Unnamed: 0.1,Unnamed: 0,MJD,RA,DEC,flux,dflux,x,y,dx,dy,sub stdv,band
0,0,0,59295.384410,270.31178,-28.7764,85973.234,3273.1940,32.397033,29.893481,0.097754,0.100575,1240.62120,g
1,1,1,59295.385860,270.31178,-28.7764,129424.810,5374.8600,32.717733,29.906466,0.082595,0.082490,550.25806,r
2,2,2,59295.386775,270.31178,-28.7764,95151.945,4527.8467,32.822846,30.286869,0.081065,0.080148,285.05103,i
3,3,3,59295.387456,270.31178,-28.7764,91161.890,5078.4950,32.809824,30.463182,0.104316,0.104172,160.19868,z
4,4,4,59295.388149,270.31178,-28.7764,88571.250,3378.8843,32.742653,29.798114,0.092633,0.095972,1212.44030,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,1307,1307,59791.094708,270.31178,-28.7764,50053.633,3274.7495,32.444530,30.462309,0.172719,0.174766,1049.39440,i
1308,1308,1308,59791.095393,270.31178,-28.7764,73972.240,5075.3364,32.177439,29.729061,0.237353,0.243898,918.32947,z
1309,1309,1309,59791.099103,270.31178,-28.7764,83124.420,5164.2427,32.687166,30.521544,0.213823,0.214336,794.15870,z
1310,1310,1310,59791.107077,270.31178,-28.7764,70128.016,3993.0544,31.918269,29.823605,0.185880,0.191620,989.34326,i
